In [1]:
from clingo.symbol import Number, String
from clingo.control import Control

In [2]:
def run(axioms):
    evASP = Control("0")
    evASP.add("base", [], axioms)

    evASP.ground([("base", [])])
    
    with evASP.solve(yield_=True) as handle:
        #print(handle)
        for m in handle: 
            print("Answer:\n{}\n".format(m))

In [3]:
code = """

causes(move, bat(B-1), bat(B)) :- bat(B), bat(B-1).
causes(move, pos(P+1), pos(P)) :- pos(P), pos(P+1).

pos(1..3).
bat(0..1).



"""

run(code)

Answer:
bat(0) bat(1) causes(move,pos(2),pos(1)) causes(move,pos(3),pos(2)) causes(move,bat(0),bat(1)) pos(1) pos(2) pos(3)



In [4]:
sokrat = """

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

causes(chargedWith(X,Y), inPrison(X), 1) :- person(X), mistake(Y).

causes(escapes(X, prison), -inPrison(X), 2) :- person(X).
cond(2, inPrison(X)) :- person(X).

causes(drinks(X,hemlock), -isAlive(X), 3) :- person(X).
cond(3, isAlive(X)) :- person(X).

impos(sentencedTo(X,Y), 4) :- person(X), sanction(Y).
cond(4, -inPrison(X)) :- person(X).

impos(walksOutside(X), 5) :- person(X).
cond(5, inPrison(X)) :- person(X).

mistake(impety).
sanction(death).
person(sokrates).

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#show holds/2.
#show causes/3.

"""

#################
meta = """
time(1..5).
holds(Head, T+1) :- causes(Act, Head, Nr), not condNotHolds(Nr, T), time(T).
condNotHolds(Nr, T) :- cond(Nr, Cond), not holds(Cond, T), time(T).

%holds(F,T+1) :- holds(F,T), not holds(-F,T+1), time(T).
%holds(-F,T+1) :- holds(-F,T), not holds(F, T+1).

% :- holds(X, T), holds(-X,T).

#show holds/2.
"""
#################


run(sokrat)

Answer:
causes(drinks(sokrates,hemlock),-isAlive(sokrates),3) causes(escapes(sokrates,prison),-inPrison(sokrates),2) causes(chargedWith(sokrates,impety),inPrison(sokrates),1)



In [ ]:
#########################################################
#########################################################
###################  LeBLanc 2  #########################

In [6]:
AD = """

impo(e1, 5).
cond(5, a).

caus(e1, e, 6).
cond(6, -e).

caus(e2, d, 7).
cond(7, -d).

caus(e3, a, 8).
cond(8, -a).

caus(e3, c, 9).
cond(9, -c).

impo(e3, 10).
cond(10, -e).

impo(e3, 11).
cond(11, -f).

if(b,12).
cond(12, c).

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

time(1..4).


"""
run(AD)

Answer:
time(1) time(2) time(3) time(4) impo(e1,5) impo(e3,10) impo(e3,11) cond(5,a) cond(12,c) cond(6,-e) cond(7,-d) cond(8,-a) cond(9,-c) cond(10,-e) cond(11,-f) caus(e1,e,6) caus(e2,d,7) caus(e3,a,8) caus(e3,c,9) if(b,12)



In [7]:
scena = """

holds(-a,1; -b,1; -c,1; -d,1; -e,1; f,1).
occ(e1, 1).

holds(-a,2;-b,2;-c,2;-d,2;e,2;f,2).
occ(e2,2).

holds(-a,3;-b,3;-c,3;d,3;e,3;f,3). 
occ(e3,3). 

holds(a,4;b,4;c,4;d,4;e,4;f,4).


"""
run(scena)

Answer:
occ(e1,1) occ(e2,2) occ(e3,3) holds(-a,1) holds(-b,1) holds(-c,1) holds(-d,1) holds(-e,1) holds(f,1) holds(-a,2) holds(-b,2) holds(-c,2) holds(-d,2) holds(e,2) holds(f,2) holds(-a,3) holds(-b,3) holds(-c,3) holds(d,3) holds(e,3) holds(f,3) holds(a,4) holds(b,4) holds(c,4) holds(d,4) holds(e,4) holds(f,4)



### Transition Event

In [31]:
theta = """
time(1..4).


% outcome
theta(a;b;c;d;e;f).

theta_transit(T) :- theta_sat(T+1), not_theta_sat(T).
theta_sat(T) :- not not_theta_sat(T), time(T).
not_theta_sat(T) :- theta(Lit), not holds(Lit,T), time(T).


%#show theta_transit/1.
%#show theta_sat/1.
%#show not_theta_sat/1.


"""

outcome_transit = scena+theta
run(outcome_transit)

Answer:
time(1) time(2) time(3) time(4) holds(-a,1) holds(-b,1) holds(-c,1) holds(-d,1) holds(-e,1) holds(f,1) holds(-a,2) holds(-b,2) holds(-c,2) holds(-d,2) holds(e,2) holds(f,2) holds(-a,3) holds(-b,3) holds(-c,3) holds(d,3) holds(e,3) holds(f,3) holds(a,4) holds(b,4) holds(c,4) holds(d,4) holds(e,4) holds(f,4) lit_transit(e,1) lit_transit(d,2) lit_transit(a,3) lit_transit(b,3) lit_transit(c,3) not_theta_sat(1) not_theta_sat(2) not_theta_sat(3) theta_sat(4) theta_transit(3) theta(a) theta(b) theta(c) theta(d) theta(e) theta(f) occ(e1,1) occ(e2,2) occ(e3,3)



### direct and indirect Effects

In [52]:
effs = """
time(1..4).
id(5..12).

dir_eff(Lit,T) :- occ(Ev, T), caus(Ev, Lit, Id), id(Id).
inert_eff(Lit,T) :- holds(Lit,T), holds(Lit,T+1),time(T).
ind_eff(Lit,T) :- holds(Lit,T+1), not dir_eff(Lit,T), not inert_eff(Lit,T).

out_dir_eff(Lit,T) :- theta(Lit), dir_eff(Lit,T).
out_ind_eff(Lit,T) :- theta(Lit), ind_eff(Lit,T).


%#show dir_eff/2.
%#show inert_eff/2.
%#show ind_eff/2.
%#show out_dir_eff/2.
%#show out_ind_eff/2.

"""
basics = outcome_transit+effs+AD
run(basics)

Answer:
time(1) time(2) time(3) time(4) holds(-a,1) holds(-b,1) holds(-c,1) holds(-d,1) holds(-e,1) holds(f,1) holds(-a,2) holds(-b,2) holds(-c,2) holds(-d,2) holds(e,2) holds(f,2) holds(-a,3) holds(-b,3) holds(-c,3) holds(d,3) holds(e,3) holds(f,3) holds(a,4) holds(b,4) holds(c,4) holds(d,4) holds(e,4) holds(f,4) lit_transit(e,1) lit_transit(d,2) lit_transit(a,3) lit_transit(b,3) lit_transit(c,3) not_theta_sat(1) not_theta_sat(2) not_theta_sat(3) theta_sat(4) theta_transit(3) theta(a) theta(b) theta(c) theta(d) theta(e) theta(f) id(5) id(6) id(7) id(8) id(9) id(10) id(11) id(12) caus(e1,e,6) caus(e2,d,7) caus(e3,a,8) caus(e3,c,9) occ(e1,1) occ(e2,2) occ(e3,3) dir_eff(e,1) dir_eff(d,2) dir_eff(a,3) dir_eff(c,3) inert_eff(-a,1) inert_eff(-b,1) inert_eff(-c,1) inert_eff(-d,1) inert_eff(f,1) inert_eff(-a,2) inert_eff(-b,2) inert_eff(-c,2) inert_eff(e,2) inert_eff(f,2) inert_eff(d,3) inert_eff(e,3) inert_eff(f,3) ind_eff(-a,0) ind_eff(-b,0) ind_eff(-c,0) ind_eff(-d,0) ind_eff(-e,0) ind_eff

### 1st Explain

In [69]:
out1 = """

#show out_dir_eff/2.
#show out_ind_eff/2.

"""

first = out1+basics
run(first)

Answer:
out_dir_eff(e,1) out_dir_eff(d,2) out_dir_eff(a,3) out_dir_eff(c,3) out_ind_eff(f,0) out_ind_eff(b,3)



### OST

In [92]:
remain_ensure =  """


lit_transit(Lit,T) :- holds(Lit, T+1), not holds(Lit, T), time(T).
%#show lit_transit/2.

lit_ensures(Lit, T1, T) :- holds(Lit, T), lit_transit(Lit,T1), T1 < T, time(T), time(T1).
% # minimize { T1 : lit_ensure(Lit,T1,T)} PROBLEM with aggregate

%#show lit_ensures/3.
%#show theta_transit/1.

o_supp(Lit, I, J) :- theta_transit(J), lit_ensures(Lit, I, J).
%#show o_supp/3.

remain(Lit,T) :- theta(Lit), not out_dir_eff(Lit,T), not out_ind_eff(Lit,T), time(T).
%#show remain/2.

o_init(Lit, T) :- remain(Lit,T), not o_supp(Lit,_,T).
%#show o_init/2.


"""
basics2 = remain_ensure+basics
run(basics2)

Answer:
time(1) time(2) time(3) time(4) holds(-a,1) holds(-b,1) holds(-c,1) holds(-d,1) holds(-e,1) holds(f,1) holds(-a,2) holds(-b,2) holds(-c,2) holds(-d,2) holds(e,2) holds(f,2) holds(-a,3) holds(-b,3) holds(-c,3) holds(d,3) holds(e,3) holds(f,3) holds(a,4) holds(b,4) holds(c,4) holds(d,4) holds(e,4) holds(f,4) lit_transit(e,1) lit_transit(d,2) lit_transit(a,3) lit_transit(b,3) lit_transit(c,3) lit_ensures(e,1,2) lit_ensures(e,1,3) lit_ensures(e,1,4) lit_ensures(d,2,3) lit_ensures(d,2,4) lit_ensures(a,3,4) lit_ensures(b,3,4) lit_ensures(c,3,4) theta_transit(3) o_supp(e,1,3) o_supp(d,2,3) theta(a) theta(b) theta(c) theta(d) theta(e) theta(f) out_dir_eff(e,1) out_dir_eff(d,2) out_dir_eff(a,3) out_dir_eff(c,3) out_ind_eff(f,0) out_ind_eff(b,3) remain(a,1) remain(b,1) remain(c,1) remain(d,1) remain(f,1) remain(a,2) remain(b,2) remain(c,2) remain(e,2) remain(f,2) remain(d,3) remain(e,3) remain(f,3) remain(a,4) remain(b,4) remain(c,4) remain(d,4) remain(e,4) remain(f,4) o_init(a,1) o_init

### 2nd Explain

In [93]:
out2 = """

#show o_supp/3.
#show o_init/2.

"""

second = out2+basics2
run(second)

Answer:
o_supp(e,1,3) o_supp(d,2,3) o_init(a,1) o_init(b,1) o_init(c,1) o_init(d,1) o_init(f,1) o_init(a,2) o_init(b,2) o_init(c,2) o_init(e,2) o_init(f,2) o_init(f,3) o_init(a,4) o_init(b,4) o_init(c,4) o_init(d,4) o_init(e,4) o_init(f,4)



### TSE

In [103]:
tse = """

prec(Lit, T) :- occ(Ev,T), caus(Ev, _, Id), cond(Id, Lit).
prec(Lit, T) :- occ(Ev,T), impo(Ev,Id), cond(Id, -Lit).

%#show prec/2.
%#show lit_ensures/3.

t_supp(Lit, I, J) :- prec(Lit, J), lit_ensures(Lit, I, J).
%#show t_supp/3.

t_init(Lit, J) :- prec(Lit, J), not t_supp(Lit, _, J).
%#show t_init/2.


"""

basics3 = tse+basics2

run(basics3)

Answer:
prec(-e,1) prec(-d,2) prec(-a,3) prec(-c,3) prec(-a,1) prec(e,3) prec(f,3) lit_ensures(e,1,2) lit_ensures(e,1,3) lit_ensures(e,1,4) lit_ensures(d,2,3) lit_ensures(d,2,4) lit_ensures(a,3,4) lit_ensures(b,3,4) lit_ensures(c,3,4) t_supp(e,1,3) t_init(-e,1) t_init(-d,2) t_init(-a,3) t_init(-c,3) t_init(-a,1) t_init(f,3)



### 3rd Explain

In [104]:
out3 = """

#show t_supp/3.
#show t_init/2.

"""

third = out3+basics3
run(third)

Answer:
prec(-e,1) prec(-d,2) prec(-a,3) prec(-c,3) prec(-a,1) prec(e,3) prec(f,3) lit_ensures(e,1,2) lit_ensures(e,1,3) lit_ensures(e,1,4) lit_ensures(d,2,3) lit_ensures(d,2,4) lit_ensures(a,3,4) lit_ensures(b,3,4) lit_ensures(c,3,4) t_supp(e,1,3) t_init(-e,1) t_init(-d,2) t_init(-a,3) t_init(-c,3) t_init(-a,1) t_init(f,3)

